# 프로젝트: Movielens 영화 추천 실습
* 🎬활용 데이터: Movielens
    * 관련 링크: https://www.tensorflow.org/datasets/catalog/movielens
    
     
* 🎬암묵적 데이터인 ratings 해석
    * 한 번이라도 들었으면 선호한다고 판단한다.
    * 많이 재생한 아티스트에 대해 가중치를 주어서 더 확실히 좋아한다고 판단한다.

    
* 🎬루브릭 
    * CSR matrix가 정상적으로 만들어졌다.
        * 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.
    * MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.
        * 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.
    * 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.
        * MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.


### 사용 라이브러리

In [31]:
# 시간 측정 
# !pip install ipython-autotime
%load_ext autotime

import os
import wget # file download 
import pandas as pd 
import numpy as np


The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 530 µs (started: 2021-02-16 19:29:16 +09:00)


### 사용 디렉토리 설정

In [15]:
# import os # 위에서 이미 import 했습니다. 

project_path = os.getenv("HOME")+'/aiffel/recommendata_iu'

if not os.path.isdir(project_path):
    os.mkdir(project_path)

else:
    print("project_path에 폴더가 이미 존재합니다.")

project_path에 폴더가 이미 존재합니다.
time: 495 µs (started: 2021-02-16 14:15:17 +09:00)


In [20]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m'

if not os.path.isdir(rating_file_path):
    os.mkdir(rating_file_path)

else:
    print("rating_file_path에 폴더가 이미 존재합니다.")

rating_file_path에 폴더가 이미 존재합니다.
time: 602 µs (started: 2021-02-16 14:16:17 +09:00)


In [24]:
url = 'http://files.grouplens.org/datasets/movielens/ml-1m.zip'

if not os.path.isfile(rating_file_path + '/ratings.dat'):
    wget.download(url, out = rating_file_path)

else:
    print("ratings.dat가 이미 존재합니다.")

ratings.dat가 이미 존재합니다.
time: 660 µs (started: 2021-02-16 14:19:53 +09:00)


In [28]:
# %cd ~/aiffel/recommendata_iu/data/ml-lm

# !unzip ml-lm.zip 

[Errno 2] No such file or directory: '/home/aiffel-dj19/aiffel/recommendata_iu/data/ml-lm'
/home/aiffel-dj19/aiffel/recommendata_iu/data
unzip:  cannot find or open ml-lm.zip, ml-lm.zip.zip or ml-lm.zip.ZIP.
time: 109 ms (started: 2021-02-16 14:20:44 +09:00)


## 데이터 전처리 
* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
* 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
* 별점을 시청횟수로 해석해서 생각하겠습니다.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

In [3]:
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
